# 用numpy实现RNN
本文针对上一个笔记[BPTT算法数学推导](./BackPropagation Through Time.ipynb)，用numpy实现了一个RNN语言模型，实验数据为Google's BigQuery上下载的longish reddit 评论，我们要用RNN来生成类似reddit的评论。大部分代码参考自wildML，在其基础上进行了包装，并融入了自己的一些理解，力求写的通俗易懂。  


## 读取依赖包

In [39]:
import csv
import nltk
import itertools
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import operator

## 定义全局变量

In [40]:
_VOCABULARY_SIZE = int(os.environ.get('VOCABULARY_SIZE', '8000'))
_HIDDEN_SIZE = int(os.environ.get('_HIDDEN_SIZE', '80'))
_NBPTT_TRUNCATE = int(os.environ.get('NBPTT_TRUNCATE', '4'))
_NEPOCH = int(os.environ.get('NEPOCH', '10'))

vocabulary_size = _VOCABULARY_SIZE
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END" 

## 读取语料库
首先是读取语料库的函数，这里直接摘抄自wildML的代码，这段代码写的十分精巧，将许多功能浓缩在一行，实在是令我叹为观止。

In [41]:
'''
读取语料库
'''
def load_corpus():
    print 'Loading CSV file...'
    with open('reddit-comments-2015-08.csv', 'rb') as f:
        reader = csv.reader(f)
        # 跳过表头
        reader.next()
        # 将整篇文档分成句子的列表
        sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
        # 为每句话加上 SENTENCE_START 和 SENTENCE_END 符号
        sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
    print 'Parsed %d sentences.'%len(sentences)
    return sentences

# 读取语料库
sentences = load_corpus()

Loading CSV file...
Parsed 79170 sentences.


将文档拆成句子列表的这一句话写的非常geek，也非常难懂
```py
sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
```
我来分析一下这句话干了哪些事情：
1. 调用nltk的`sent_tokenize`对段落x[0]分句，将其转变为句子的列表，x[0]表示csv文件里一行的内容
2. 用列表生成式遍历csv文件每一行，生成句子列表的嵌套列表
3. 用星号`*`对这个嵌套列表解包，将其变为一个一个的列表
4. 用`chain()`函数将这些列表合并为一个列表  

`chain()`函数的定义如下：

```py
def chain(*iterables):
    # chain('ABC', 'DEF') --> A B C D E F
    for it in iterables:
        for element in it:
            yield element
```            

它的作用是将多个迭代器作为参数, 但只返回单个迭代器, 它产生所有参数迭代器的内容, 就好像他们是来自于一个单一的序列。  
最后再用一个列表生成式为每一句话加上起始符和结束符。
```py
sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
```

## 文本的预处理
文本预处理主要完成了：
1. 对句子的分词
2. 统计词频
3. 获取高频词，建立索引
4. 替换未登录词

In [42]:
'''
文本预处理
'''
def preprocessing(sentences):
    print 'Preprocessing...'
    # 对每个句子进行分词
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
    # 统计词频
    word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    print "Found %d unique words tokens." % len(word_freq.items())
    # 获得词频最高的词，并建立索引到词、词到索引的向量
    vocab = word_freq.most_common(vocabulary_size-1)
    index_to_word = [x[0] for x in vocab]
    index_to_word.append(unknown_token)
    word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

    print "Using vocabulary size %d." % vocabulary_size
    print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])                 
    # 将未登录词替换为unknown token
    for i, sent in enumerate(tokenized_sentences):
        tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    return tokenized_sentences, word_to_index, index_to_word

# 预处理
tokenized_sentences, word_to_index, index_to_word = preprocessing(sentences)

Preprocessing...
Found 65751 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'devoted' and appeared 10 times.


1. 对句子分词
`nltk.word_tokenize(sent)`作用是对输入的句子进行分词，输出为分词结果组成的列表。`tokenized_sentences`是一个嵌套列表，列表中的每一个元素表示一条句子的分词结果。
2. 统计词频
首先用`*`将`tokenized_sentences`分解为多个list，接着调用`itertools.chain()`函数将这些list接合成一个list，作为`nltk.FreqDist()`函数的输入，返回一个字典，键为word，值为该word的词频。
3. 获取高频词，建立索引
用`most_common`函数获取频率最高的`vocabulary_size-1`个词，返回值`vocab`是一个tuple的list，其格式如下：
```py
[(',', 18713), ('the', 13721), ('.', 6862), ('of', 6536), ('and', 6024),
('a', 4569), ('to', 4542), (';', 4072), ('in', 3916), ('that', 2982)]
```
接着我们遍历这个list，将高频词从每个tuple取出来单独构成一个list，并在最后加入unknown_token表示未登录词：
```py
index_to_word = [x[0] for x in vocab]
```
然后我们要建立词的索引，`enumerate()`函数接收一个list，并为list中的每个元素生成一个序号。一种较为naive的写法是:

```py
i = 0
for item in iterable:
    print i, item
    i += 1
```

而使用enumerate我们可以将代码简化为：

```py
for i, item in enumerate(iterable):
    print i, item
```    

接着我们生成一个由词和索引构成的tuple组成的list，并用它生成一个dict `word_to_index`。

4.替换未登录词
这里使用了一种较为高级的列表生成式：
```py
[w if w in word_to_index else unknown_token for w in sent]
```
在生成列表时候，我们可以加入条件判断以完成复杂的表达式赋值，这里如果w出现在索引字典中则保持不变，否则将被替换为unknown_token。

## 生成训练数据
上一步中，我们获得了分词结果`tokenized_sentences`和词索引`word_to_index`，这一节我们利用它们来生成训练集。

In [43]:
'''
生成训练数据
'''
def gen_train_data(tokenized_sentences, word_to_index):
    print 'Generating train data...'
    # 创建训练数据
    X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
    y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])    
    print "Generated %d training instances."%X_train.shape[0]
    print "Example input: %s"%X_train[0]
    print "Example output: %s"%y_train[0]
    return X_train, y_train

# 生成训练数据
X_train, y_train = gen_train_data(tokenized_sentences, word_to_index)

Generating train data...
Generated 79170 training instances.
Example input: [0, 6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2]
Example output: [6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2, 1]


`gen_train_data()`函数首先读取语料库，接着对句子进行了预处理，最后生成训练数据集。`X_train`和`y_train`都是二维numpy数组，在初始化这两个数组时我们采用了含有两层嵌套循环的列表生成式：
```py
    X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
    y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])    
```

第一层循环遍历每一条句子的分词序列，第二层循环遍历序列中的每个词将它们转化为索引。
注意到`sent[:-1]`代表`[sentence_start_token x ]`，`sent[1:]`代表`[x sentence_end_token]`，这是由于我们这个任务是训练一个语言模型，每次预测下一个出现的词，因此`y_train`中每个词对应`X_train`中下一个出现的词，即$y_t=x_{t+1}$。

## 实现softmax函数

In [44]:
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

解释一下为什么要减去max(x)。由于x可能会很大，exp以后可能会上溢出（overflow），而softmax是一个比值，因此如果我们在分子分母上同时乘/除以一个常数是不会影响最终结果的，我们只要同时除以一个较大的数就可以防止overflow。显然这里除以最大值是最好的，因为我们无法确定x的数量级，而最大值是关于x数量级自适应增长的。 

## RNN实现

In [45]:

class RNNNumpy:
    def __init__(self, vocab_size, hidden_size = 100, bptt_truncate = 4):
        # 初始化模型超参数
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.bptt_truncate = bptt_truncate
        # 输入权重矩阵， H x K 矩阵
        self.W_in = np.random.uniform(-np.sqrt(1.0/vocab_size), np.sqrt(1.0/vocab_size), (hidden_size, vocab_size))
        # 隐层权重矩阵， H x H 矩阵
        self.W_rec = np.random.uniform(-np.sqrt(1.0/hidden_size), np.sqrt(1.0/hidden_size), (hidden_size, hidden_size))
        # 输出权重矩阵， K x H 矩阵
        self.W_out = np.random.uniform(-np.sqrt(1.0/hidden_size), np.sqrt(1.0/hidden_size), (vocab_size, hidden_size))


    def forward_propagation(self, x):
        '''前向传播
        # 参数：
            x：输入句子序列
            y：输出句子序列
        # 返回值：
            h：各时刻隐藏层的输出
            o：各时刻softmax层的输出
        '''
        # 句子长度
        T = len(x)
        # 隐层各时刻输出，多加一列用于存放全0向量，以表示h[-1]
        h = np.zeros((T+1, self.hidden_size))
        # 各时刻softmax的输出
        o = np.zeros((T, self.vocab_size))
        # 计算各时刻隐单元值和输出
        for t in np.arange(T):
            h[t] = np.tanh(self.W_rec.dot(h[t-1]) + self.W_in[:, x[t]])
            o[t] = softmax(self.W_out.dot(h[t]))
        return h, o

    def calc_total_loss(self, X_train, y_train):
        '''计算总交叉熵损失
        # 参数
            X_train：训练集特征
            y_train：训练集标签
        # 返回值：
            E：训练集上的交叉熵
        '''
        E = 0
        for x, y in zip(X_train, y_train):
            h, o = self.forward_propagation(x)
            correct_word_predictions = o[np.arange(len(y)), y]
            E += -1 * np.sum(np.log(correct_word_predictions))
        return E

    def calc_avg_loss(self, X_train, y_train):
        '''计算每个词上的平均交叉熵损失
        '''
        num_total_words = np.sum((len(y_i) for y_i in y_train))
        return  self.calc_total_loss(X_train, y_train)/ num_total_words

    def bptt(self, x, y):
        '''bptt：随时间反向传播
        # 参数
            x：输入句子序列
            y：输出句子序列
        # 返回值
            W_in_grad：交叉熵关于输入矩阵的梯度
            W_rec_grad：交叉熵关于隐层矩阵的梯度
            W_out_grad：交叉熵关于输出矩阵的梯度
        '''
        T = len(x)
        W_in_grad = np.zeros(self.W_in.shape)
        W_rec_grad = np.zeros(self.W_rec.shape)
        W_out_grad = np.zeros(self.W_out.shape)
        
        h, o = self.forward_propagation(x)
        residual = o
        residual[np.arange(T), y] -= 1
        for t in np.arange(T)[::-1]:
            W_out_grad += np.outer(residual[t], h[t])
            delta_k = self.W_out.T.dot(residual[t]) * (1 - h[t] ** 2)
            for k in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                W_rec_grad += np.outer(delta_k, h[k-1])
                W_in_grad[:, x[k]] += delta_k
                delta_k = self.W_rec.T.dot(delta_k) * (1 - h[k-1]**2)
        return (W_in_grad, W_rec_grad, W_out_grad)

    def sgd(self, x, y, learning_rate):
        '''随机梯度下降
        # 参数：
            x：输入句子序列
            y：输出句子序列 
            learning_rate: 学习率     
        '''
        W_in_grad, W_rec_grad, W_out_grad = self.bptt(x, y)
        self.W_in -= learning_rate * W_in_grad
        self.W_rec -= learning_rate * W_rec_grad
        self.W_out -= learning_rate * W_out_grad             
    
    def train_with_sgd(self, X_train, y_train, nb_epoch = 100, learning_rate = 0.005, evaluate_loss_after = 5):
        '''用随机梯度下降训练模型
        # 参数：
            X_train:训练集特征
            y_train:训练集标签
            nb_epoch:轮询次数
            learning_rate:学习率
            evaluate_loss_after:设定经过多少轮epoch后计算损失函数
        '''
        losses = []
        for epoch in range(nb_epoch):
            if epoch % evaluate_loss_after ==0:
                loss = self.calc_avg_loss(X_train, y_train)
                losses.append(loss)
                if len(losses) > 1 and losses[-1] > losses[-2]:
                    learning_rate *= 0.5
                    print "Setting learning rate to %f"%learning_rate
                print time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time())) + \
                " epoch=%d, num_samples_seen=%d, loss=%f"%(epoch, epoch * X_train.shape[0], loss)
            for x, y in zip(X_train, y_train):
                self.sgd(x, y, learning_rate)

    def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
        '''梯度检查
        # 参数：
        h: 参数增量
        error_threshold: 误差阈值
        '''
        # 用反向传播计算梯度，我们希望检查这些梯度是否正确
        bptt_gradients = self.bptt(x, y)
        # 我们要检查的参数名列表
        model_parameters = ['W_in', 'W_rec', 'W_out']
        # 为每个参数进行梯度检查
        for pidx, pname in enumerate(model_parameters):
            # 获得参数的实际值，比如model.W_in
            parameter = operator.attrgetter(pname)(self)
            print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
            # 遍历参数矩阵的每个元素，比如 (0,0), (0,1), ...
            it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                # 保存原来的值以便我们可以恢复
                original_value = parameter[ix]
                # 用公式(f(x+h) - f(x-h))/(2*h)估计梯度
                parameter[ix] = original_value + h
                gradplus = self.calc_total_loss([x],[y])
                parameter[ix] = original_value - h
                gradminus = self.calc_total_loss([x],[y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                # 恢复参数原来的值
                parameter[ix] = original_value
                # 反向传播计算得到的梯度值
                backprop_gradient = bptt_gradients[pidx][ix]
                # 计算相对误差: (|x - y|/(|x| + |y|))
                relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
                # 如果误差高于阈值，则检查失败
                if relative_error > error_threshold:
                    print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                    print "+h Loss: %f" % gradplus
                    print "-h Loss: %f" % gradminus
                    print "Estimated_gradient: %f" % estimated_gradient
                    print "Backpropagation gradient: %f" % backprop_gradient
                    print "Relative Error: %f" % relative_error
                    return
                it.iternext()
            print "Gradient check for parameter %s passed." % (pname)

## 梯度检查
上一节我们已经实现了RNN，包括前馈、随时间反向传播、随机梯度下降和计算损失函数等功能，但我们还不知道我们的实现是否正确，因此需要用梯度检查（Gradient Check）技术来检查bptt返回的梯度与估计的梯度是否一致。梯度检查的原理是用损失函数来估计梯度，具体方法如下：
$$\bigg|\frac{\partial E}{\partial \theta} - \lim_{h\to 0}\frac{f(\theta+h)-f(\theta-h)}{2h}\bigg|<\epsilon$$
其中$\epsilon>0$是一个误差阈值。如果两个值的绝对值误差不超过这个阈值，则梯度检查通过；否则就是我们的实现出了问题。  
对于求导变量是矩阵的情况，我们需要对矩阵中的每一个元素都进行梯度检查，如果我们直接用语料库数据去做梯度检查，则时间开销会很高。实际上，我们只要构造一个简单的测试数据集就可以解决这个问题：

In [46]:
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, 1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter W_in with size 1000.
Gradient check for parameter W_in passed.
Performing gradient check for parameter W_rec with size 100.
Gradient check for parameter W_rec passed.
Performing gradient check for parameter W_out with size 1000.
Gradient check for parameter W_out passed.


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:150: RuntimeWarning: invalid value encountered in double_scalars


梯度检查的好处在于，它可以同时检查前馈、反向传播和计算损失函数等功能的正确性，而这些功能正是神经网络算法的核心组件。

## 训练模型
首先，我们估计一下一次SGD的时间

In [47]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
%timeit model.sgd(X_train[10], y_train[10], 0.005)

1 loops, best of 3: 334 ms per loop


我们看到，没有GPU加速的SGD一次要花大约300ms，而我们有将近80000个样本，也就是说一次epoch要将近6个小时，训练好一个RNN估计要好几个星期了。为了能尽快训练一个模型，我们可以对训练集进行裁剪，只使用其最开始的100个样本：

In [48]:
model = RNNNumpy(_VOCABULARY_SIZE, _HIDDEN_SIZE, _NBPTT_TRUNCATE)
model.train_with_sgd(X_train[:100], y_train[:100], nb_epoch = _NEPOCH, evaluate_loss_after = 1)

2016-08-23 10:33:50 epoch=0, num_samples_seen=0, loss=8.987291
2016-08-23 10:34:05 epoch=1, num_samples_seen=100, loss=8.975247
2016-08-23 10:34:20 epoch=2, num_samples_seen=200, loss=8.956812
2016-08-23 10:34:37 epoch=3, num_samples_seen=300, loss=8.841700
2016-08-23 10:34:52 epoch=4, num_samples_seen=400, loss=6.822989
2016-08-23 10:35:07 epoch=5, num_samples_seen=500, loss=6.306852
2016-08-23 10:35:24 epoch=6, num_samples_seen=600, loss=6.048093
2016-08-23 10:35:41 epoch=7, num_samples_seen=700, loss=5.877046
2016-08-23 10:36:01 epoch=8, num_samples_seen=800, loss=5.746562
2016-08-23 10:36:20 epoch=9, num_samples_seen=900, loss=5.644863


## 生成句子
在生成句子时，我们每次从输出概率的多项式分布中采样，产生下一个词。为什么要从多项式分布进行采样？我觉得原因是增加随机性，防止每次产生同一条句子。另外，我发现如果将采样改为每次取概率最大的那个词，会出现每次预测同一个词的情况，比如"i the the the the......"，这可能是由于优化过程陷入了局部最优，因此需要尝试其他优化方法。

In [49]:
def gen_sentence(model):
    '''生成句子
    '''
    new_sentence = [word_to_index[sentence_start_token]]
    while new_sentence[-1] != word_to_index[sentence_end_token]:
        h, next_word_probs = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence = [index_to_word[word] for word in new_sentence[1:-1]]
    return sentence

num_sentences = 20
sentence_min_len = 2
for i in xrange(num_sentences):
    while True:
        sentence = gen_sentence(model)
        if len(sentence) > sentence_min_len:
            break
    sentence = ' '.join([word.encode('utf-8') for word in sentence])
    print '=== sentence {} ==='.format(i + 1)
    print sentence

=== sentence 1 ===
sharking objectively regions ease : films i like shared .
=== sentence 2 ===
is qb played local alive gatherer would the and ; it and is son a surprisingly logic and ; the what why device require rest only off-topic having is he bad zero having a paid for ranking sell places a she ? gt and decide 2faskreddit what .
=== sentence 3 ===
gifts lifted missing gauge .
=== sentence 4 ===
abandoned languages players i .
=== sentence 5 ===
^^^message ? the but .
=== sentence 6 ===
price magic efficient load ? the and of is always missing rights calm it the cab going the the 250 have the stressful have usual event the custom the do players indeed unable 's me the travelling was . anything than ; the users and attempts defense a into i house simplistic wrote house sound or
=== sentence 7 ===
update interesting and convince
=== sentence 8 ===
proposed enter would of the not why is , to site and is from article dodging suggests very early her but a n't .
=== sentence 9 ===
empire